# PixelFrame engineering design demo

To do

Shear from steel fiber.

Import required packages

In [1]:
using Pkg
Pkg.activate("..");
Pkg.status()
Pkg.instantiate()

  Activating project at `~/dev/ConcreteExplorer`


Status `~/dev/ConcreteExplorer/Project.toml`
  [acfe5cae] AsapSections v0.1.0 `https://github.com/keithjlee/AsapSections#main`
  [336ed68f] CSV v0.10.12
  [13f3f980] CairoMakie v0.11.5
  [a93c6f00] DataFrames v1.6.1
  [59287772] Formatting v0.4.2
  [f6369f11] ForwardDiff v0.10.36
  [e9467ef8] GLMakie v0.9.5
  [682c06a0] JSON v0.21.4
  [ee78f7c6] Makie v0.20.4
  [01bcebdf] Nonconvex v2.1.3
  [d3d89cbb] NonconvexMMA v1.0.0
  [b43a31b8] NonconvexNLopt v0.1.8
  [429524aa] Optim v1.7.8
⌃ [7f7a1694] Optimization v3.20.2
⌃ [4e6fcdb7] OptimizationNLopt v0.1.8
  [90137ffa] StaticArrays v1.9.1
  [e88e6eb3] Zygote v0.6.68
  [10745b16] Statistics v1.9.0
Info Packages marked with ⌃ have new versions available and may be upgradable.


In [48]:
using CSV
using DataFrames
using JSON
using Dates
using ProgressBars

Load the precalculated catalog

In [4]:
pwd()

"/Users/pitipatwongsittikan/dev/ConcreteExplorer/src"

In [31]:
catalog = CSV.read("Catalogs/test_catalog.csv", DataFrame); #load the pre-calc catalog
sort!(catalog, [:carbon, :fc′, :as, :ec])


println("The catalog was sorted by ascending order from:\ncarbon -> fc′ -> as -> ec")
println(catalog[1:20,:])

The catalog was sorted by ascending order from:
carbon -> fc′ -> as -> ec
20×9 DataFrame
 Row │ fc′      as       ec       fpe      Pu       Mu       Vu       carbon   ID    
     │ Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Int64 
─────┼───────────────────────────────────────────────────────────────────────────────
   1 │    28.0     99.0     0.05      0.0  627.772  109.659  8.14936  16.6541      1
   2 │    28.0     99.0     0.05     46.5  625.378  108.641  8.14928  16.6541      2
   3 │    28.0     99.0     0.05     93.0  622.985  107.706  8.1492   16.6541      3
   4 │    28.0     99.0     0.05    139.5  620.591  106.846  8.14912  16.6541      4
   5 │    28.0     99.0     0.05    186.0  618.197  106.032  8.14904  16.6541      5
   6 │    28.0     99.0     0.05    232.5  615.803  108.382  8.14896  16.6541      6
   7 │    28.0     99.0     0.05    279.0  613.409  110.748  8.14887  16.6541      7
   8 │    28.0     99.0     0.05    325.5  611.016  113.11

Load the demand file   

In [41]:
#load demands into a dictionary
demand_path = joinpath(@__DIR__, "Demands/test_input_CISBAT_dataset.json");
open(demand_path, "r") do f
    global demands = DataFrame(JSON.parse(f, dicttype=Dict{String,Any}))
    ns = size(demands)[1]
    demands[!,:idx] = 1:ns
    println("Demands were loads from:\n", demand_path)
end
println(demands[1:10,:])

Demands were loads from:
/Users/pitipatwongsittikan/dev/ConcreteExplorer/src/Demands/test_input_CISBAT_dataset.json
10×8 DataFrame
 Row │ vu        s_idx  pu            e_idx  type     ec_max     mu       idx   
     │ Float64   Int64  Float64       Int64  String   Float64    Float64  Int64 
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ 129.755       0  -1.4779e-15       0  primary  0.0423597  129.755      1
   2 │ 129.755       1  -7.38948e-16      0  primary  0.117666   194.632      2
   3 │ 129.755       2  -7.38948e-16      0  primary  0.180421   259.509      3
   4 │  45.9809      3   1.4779e-15       0  primary  0.230625   283.246      4
   5 │  45.9809      4   7.38948e-16      0  primary  0.268278   306.237      5
   6 │  45.9809      5  -7.38948e-16      0  primary  0.29338    329.227      6
   7 │  45.9809      6  -7.38948e-16      0  primary  0.305931   352.218      7
   8 │  45.9809      7   0.0              0  primary  0.305931   3

The demand catalog was labeled with 0 index scheme, add 1 so they are compatible with Julia 1 index scheme.

In [42]:
@show old_min_e_idx = minimum(demands[!, "e_idx"]);
@show old_min_s_idx = minimum(demands[!, "s_idx"]);

@show old_max_e_idx = maximum(demands[!, "e_idx"]);
@show old_max_s_idx = maximum(demands[!, "s_idx"]);



old_min_e_idx = minimum(demands[!, "e_idx"]) = 0
old_min_s_idx = minimum(demands[!, "s_idx"]) = 0
old_max_e_idx = maximum(demands[!, "e_idx"]) = 29
old_max_s_idx = maximum(demands[!, "s_idx"]) = 13


In [43]:
demands[!,"e_idx"] .+= 1 ;
demands[!,"s_idx"] .+= 1 ;

In [44]:
@show new_min_e_idx = minimum(demands[!, "e_idx"]);
@show new_min_s_idx = minimum(demands[!, "s_idx"]);

@show new_max_e_idx = maximum(demands[!, "e_idx"]);
@show new_max_s_idx = maximum(demands[!, "s_idx"]);


new_min_e_idx = minimum(demands[!, "e_idx"]) = 1
new_min_s_idx = minimum(demands[!, "s_idx"]) = 1
new_max_e_idx = maximum(demands[!, "e_idx"]) = 30
new_max_s_idx = maximum(demands[!, "s_idx"]) = 14


Fix redundant element indices due to Primary and Secondary labels

In [45]:
global e_idx = 1 
for i in 1:size(demands)[1]
    if i !=1
        demands[i-1, :e_idx] = e_idx
        if demands[i,:s_idx] < demands[i-1,:s_idx]
            global e_idx +=1 
        end
        
    end
    if i == size(demands)[1]
        demands[i, :e_idx] = e_idx
    end
end

Create a function to filter only feasible designs in a demand point

In [46]:

"""
filter feasible demands
"""
function filter_demands(demands::DataFrame, catalog::DataFrame)

    ns = size(demands)[1]          #total number of sections
    ne = unique(demands[!, :e_idx]) #total number of elements
    nc = size(catalog, 1)           #total number of available choices

    output_results = Dict{Int64,Vector{Int64}}() #map between demands and indices of the feasible section.
    demands[!, "total_results"] = zeros(Int64, size(demands)[1])
    #go through each section and filter the feasible designs from the catalog.
    for i = 1:ns
        en = demands[i, "e_idx"]
        sn = demands[i, "s_idx"]
        # push!(elements_to_sections[en], sn)

        pu = demands[i, "pu"]
        mu = demands[i, "mu"]
        # vu = demands[i,"vu"]
        ec_max = demands[i, "ec_max"]

        global feasible_sections = filter([:Pu, :Mu, :Vu, :ec] => (x1, x2, x3, x4) ->
                x1 > pu &&
                x2 > mu &&
                # x3 > vu && Have not consider the shear
                x4 <= ec_max * 1000,
            catalog
        )

        if size(feasible_sections)[1] == 0 #if the number of feasible results = 0
            println(feasible_sections[!, :ID])
            println("section $sn: element $en")
            output_results[i] = [0]
            demands[i, "total_results"] = 0
            # println(outr)
        else
            output_results[i] = feasible_sections[!, :ID]
            demands[i, "total_results"] = length(feasible_sections[!, :ID])
            # println(outr)
        end
    end

    return output_results
end


filter_demands

In [49]:
"""
Find the optimum result for each element.
    
    !! not optimum
For the same element, will use the same fc′ steel size and post tensioning stress.

"""
function find_optimum(output_results, demands)
    ns = size(demands)[1]
    ne = unique(demands[!, :e_idx])

    elements_designs = Dict(k => [[]] for k in unique(demands[!, "e_idx"])) #element index to list of designs
    elements_to_sections = Dict(k => Int[] for k in unique(demands[!, "e_idx"]))
    for i = 1:ns
        en = demands[i, "e_idx"]
        # sn = demands[i, "s_idx"]
        push!(elements_to_sections[en], i)
    end

    #find fc′, as, and fpe that appear in all sections in an element.
    for i in ProgressBars(ne) #loop each element
        println("working on element $i out of $(length(ne)) elements")
        sections = elements_to_sections[i] #sections associated with this element
        local ns = length(sections)     # number of sections in this element 

        #start from the middle-ish section
        mid = div(ns, 2) #note that section is in the form of 1 to ns, consecutively.

        #get the feasible designs for the middle section
        feasible_idx = output_results[sections[mid]]

        #catalog was already sorted, so I think we can leave this part, just filter, to save time.
        sub_catalog = sort(catalog[feasible_idx, :], [:carbon, :fc′, :ec])
        # sub_catalog = catalog[feasible_idx, :]

        #now, loop each design in the sub catalog, see if as and fpe are available in all sections.
        #if not, remove that design from the sub catalog.
        #if yes, keep it.

        for d in eachrow(sub_catalog)
            # all_fc′ = true
            all_as = true
            all_fpe = true
            for s in sections
                #check if the design is available in that section.
                #if not, remove it from the sub catalog.
                #if yes, keep it.
                # if !(d[:fc′] ∈ catalog[output_results[s], :fc′])
                #     all_fc′ = false
                # end
                if !(d[:as] ∈ catalog[output_results[s], :as])
                    all_as = false
                end
                if !(d[:fpe] ∈ catalog[output_results[s], :fpe])
                    all_fpe = false
                end
            end
            if all_as && all_fpe
                #first run, found, move on.
                break
            end
        end

        # sort!(sub_catalog, [:carbon, :fpe, :as, :ec])

        #get the first one, they will appear in the entire thing anyway.
        this_fpe = sub_catalog[1, :fpe]
        this_as = sub_catalog[1, :as]

        sections_designs = Vector{Vector}(undef, ns)
        for is in eachindex(elements_to_sections[i])
            #current section index
            s = elements_to_sections[i][is]

            feasible_idx = output_results[s]
            sub_catalog = catalog[feasible_idx, :]

            fpe_as(fpe::Float64, as::Float64) = fpe == this_fpe && as == this_as

            this_catalog = filter([:fpe, :as] => fpe_as, catalog[output_results[s], :])

            sort!(this_catalog, [:carbon, :ec])



            #get the first one, it's the best.
            select_ID = this_catalog[1, :ID]
            #find lowest e for this one.
            sections_designs[is] = collect(catalog[select_ID, :])
            println(sections_designs[is])
        end

        elements_designs[i] = sections_designs

    end

    return elements_designs
end

find_optimum

In [47]:
output_results = filter_demands(demands,catalog)


Dict{Int64, Vector{Int64}} with 552 entries:
  56  => [16, 17, 18, 19, 20, 21, 22, 23, 24, 25  …  21799, 21800, 21801, 21802…
  35  => [975, 976, 977, 978, 979, 1003, 1004, 1005, 1006, 1007  …  21799, 2180…
  425 => [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  21799, 21800, 21801, 21802, 21803, …
  429 => [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  21799, 21800, 21801, 21802, 21803, …
  60  => [6658, 6659, 6660, 6661, 6686, 6687, 6688, 6689, 6690, 6691  …  21799,…
  220 => [134, 135, 136, 160, 161, 162, 163, 164, 165, 166  …  21799, 21800, 21…
  308 => [45, 46, 47, 48, 49, 50, 51, 52, 53, 54  …  21799, 21800, 21801, 21802…
  67  => [6658, 6659, 6660, 6661, 6686, 6687, 6688, 6689, 6690, 6691  …  21799,…
  215 => [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  21799, 21800, 21801, 21802, 21803, …
  73  => [569, 570, 571, 572, 573, 596, 597, 598, 599, 600  …  21799, 21800, 21…
  319 => [45, 46, 47, 48, 49, 50, 51, 52, 53, 54  …  21799, 21800, 21801, 21802…
  251 => [132, 133, 134, 135, 136, 137, 138, 139, 140, 141  …  2